In [2]:
# Gerekli Kütüphaneleri Ekliyoruz. Numpy ve Pandas
import numpy as np
import pandas as pd


In [3]:
# Hava durumu , sıcaklık nem ve rüzgar gibi koşulların karşısında tenis oynama 
hava=['Güneşli','Güneşli','Bulutlu','Yağmurlu','Yağmurlu','Yağmurlu','Bulutlu','Güneşli','Güneşli','Yağmurlu','Güneşli','Bulutlu','Bulutlu','Yağmurlu']
sıcaklık=['Sıcak','Sıcak','Sıcak','Ilık','Soğuk','Soğuk','Soğuk','Ilık','Soğuk','Ilık','Ilık','Ilık','Sıcak','Ilık']
nem=['Yüksek','Yüksek','Yüksek','Yüksek','Normal','Normal','Normal','Yüksek','Normal','Normal','Normal','Yüksek','Normal','Yüksek']
rüzgar=['Hafif','Güçlü','Hafif','Hafif','Hafif','Güçlü','Güçlü','Hafif','Hafif','Hafif','Güçlü','Güçlü','Hafif','Güçlü']
durum=np.array(["Hayır","Hayır","Evet","Evet","Evet","Hayır","Evet","Hayır","Evet","Evet","Evet","Evet","Evet","Hayır"])
tenis_verisi = pd.DataFrame([hava,sıcaklık,nem,rüzgar,durum])
tenis_verisi = tenis_verisi.T
tenis_verisi.columns=['Hava','Sıcaklık','Nem','Rüzgar','Durum']
tenis_verisi

,Hava,Sıcaklık,Nem,Rüzgar,Durum
0,Güneşli,Sıcak,Yüksek,Hafif,Hayır
1,Güneşli,Sıcak,Yüksek,Güçlü,Hayır
2,Bulutlu,Sıcak,Yüksek,Hafif,Evet
3,Yağmurlu,Ilık,Yüksek,Hafif,Evet
4,Yağmurlu,Soğuk,Normal,Hafif,Evet
5,Yağmurlu,Soğuk,Normal,Güçlü,Hayır
6,Bulutlu,Soğuk,Normal,Güçlü,Evet
7,Güneşli,Ilık,Yüksek,Hafif,Hayır
8,Güneşli,Soğuk,Normal,Hafif,Evet
9,Yağmurlu,Ilık,Normal,Hafif,Evet


In [4]:
class agac:
  def __init__(self,veri):
    '''
    Değişkenler : veri (dataframe)*
    
    *Son kolon durumu belirtmelidir. Durum "Evet" veya 1, "Hayır" veya 0 olabilir.
    
    Sınıfın çağırılmasıyla ilk parametreler oluşturulur ve gövde fonksiyonu çağırılır.
    '''
    self.veri=veri 
    self.veri[self.veri.columns[-1]] = self.veri[self.veri.columns[-1]].replace("Evet",1)
    self.veri[self.veri.columns[-1]] = self.veri[self.veri.columns[-1]].replace("Hayır",0)
    self.uzunluk=len(veri)
    self.sonuclar=[]
    self.gövde(self.veri,self.sonuclar)
    

  def gini_skoru(self,pozitif,uzunluk):
    '''
    Değişkenler : pozitif(int)* , uzunluk(int)
    
    Dönüş : Gini Skoru
    
    * Olumlu durumların sayısı
    
    '''
    
    return (1-((pozitif/uzunluk)**2)-(((uzunluk-pozitif)/uzunluk)**2))



  def dal(self,veri):
    '''
    Değişkenler : veri (dataframe)*
    
    Dönüş : sonuc_verisi(dataframe)**
    
    * Son kolon durumu belirtmelidir. Durum değer olarak : "Evet" veya 1, "Hayır" veya 0 olabilir.
    ** Verideki özelliklerin gini skor değerlerini döner.
    
    '''
    veri_uzunluk=len(veri)

    sonuclar=[]
    
    for columns in veri.columns[0:-1]:

      dizi=veri[columns]

      kolon_gini=0
     
      # Loop over columns features
      for ozellik in np.unique(dizi):
        
        # Create new feature dataframe 
        ozellikler=veri[(veri[columns] == ozellik)]
        ozellikler=ozellikler.iloc[:,1:]
        ozellikler.reset_index(inplace=True,drop=True)
        
        # Get all feature decision values
        karar=ozellikler[ozellikler.columns[-1]]
        
        # Sum all feature decision values
        pozitif=sum(karar)
        
        # Item instance
        ozellik_uzunluk=len(ozellikler)

        # Calculate feature gini
        ozellik_gini_skoru=self.gini_skoru(pozitif,ozellik_uzunluk)
        
        # Calculate columns gini
        kolon_gini+=ozellik_gini_skoru*(ozellik_uzunluk/veri_uzunluk)
      
      # Add results to list
      sonuclar.append(kolon_gini)
      
    self.sonuc_verisi=pd.DataFrame([sonuclar])
    self.sonuc_verisi.columns = [columns for columns in veri.columns[0:-1]]
    
    return self.sonuc_verisi
    
  def gövde(self,veri,sonuclar):
    '''
    Değişkenler : veri (dataframe)* , sonuclar(array)
    
    * Son kolon durumu belirtmelidir. Durum değer olarak : "Evet" veya 1, "Hayır" veya 0 olabilir.
    
    '''

   
    dal=self.dal(veri)

    # Get minimum gini index 
  
    kucuk_gini=dal.idxmin(axis=1).values[0]


    # Loop over minimum gini index feature
    for indeks,ozellik in enumerate(np.unique(veri[kucuk_gini])):

        # Create new dataframe for feature
        veri_dizesi=veri[(veri[kucuk_gini] == ozellik)]
        veri_dizesi=veri_dizesi.iloc[:,1:]
        veri_dizesi.reset_index(inplace=True,drop=True)


        # Add minimum gini index features
        self.sonuclar.append(kucuk_gini)
        self.sonuclar.append(ozellik)

        # Get all decision values
        karar=veri_dizesi[veri_dizesi.columns[-1]]

        # Check all decision values  
        if (karar == 1).all():
          self.sonuclar.append("1")
        elif (karar== 0).all():
          self.sonuclar.append("0")
        else:

          # Tree branch with new feature frame
          self.gövde(veri_dizesi,sonuclar)

    

In [5]:
tenis_karar_agacı=agac(tenis_verisi)
tenis_karar_agacı.sonuclar

['Hava',
 'Bulutlu',
 '1',
 'Hava',
 'Güneşli',
 'Nem',
 'Normal',
 '1',
 'Nem',
 'Yüksek',
 '0',
 'Hava',
 'Yağmurlu',
 'Rüzgar',
 'Güçlü',
 '0',
 'Rüzgar',
 'Hafif',
 '1']